In [1]:
import os
import numpy as np
import pandas as pd
import mne
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [29]:
'''
folder_path = "D:\STUDY\SEM2 3rd-year\BCI\PJ\data2a"
# Use the os module to get a list of all .gdf files in the folder
file_list = [file for file in os.listdir(folder_path) if file.endswith('T.gdf')]
file_list
# Iterate over the file list and read each .gdf file using MNE
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    raw = mne.io.read_raw_gdf(file_path, preload= True, verbose=0, )
    # Process the raw data as needed
    print(raw.info)  # Example: printing the raw data info
    # Additional processing or analysis with the raw data can be done here
raw_list = []
for f in file_list:
    raw = mne.io.read_raw_gdf(os.path.join(folder_path , f), preload=True, verbose=0,eog=['EOG-left', 'EOG-central', 'EOG-right'])
    raw_list.append(raw)

# Merge raw objects
raw = mne.concatenate_raws(raw_list)
'''

SyntaxError: EOF while scanning triple-quoted string literal (2288125970.py, line 19)

In [2]:
raw.pick_types(eeg=True,eog=False)

NameError: name 'raw' is not defined

In [3]:

DATA_DIR = 'D:\STUDY\SEM2 3rd-year\BCI\PJ\data2a'

def get_data_path(subject_no, test_or_train):
    data_type_code = 'T' if test_or_train == 'train' else 'E'
    return os.path.join(DATA_DIR, 'A0{}{}.gdf'.format(subject_no, data_type_code))


In [4]:
LAPLACIAN_NEIGHBORS = {
    'Fz': ['4'],
    '2': ['3', 'C3'],
    '3': ['2', '4', '9'],
    '4': ['Fz', '3', '5', 'Cz'],
    '5': ['4', '6', '11'],
    '6': ['5', 'C4'],
    '7': ['C3'],
    'C3': ['7', '2', '9', '14'],
    '9': ['C3', '3', 'Cz', '15'],
    'Cz': ['9', '4', '11', '16'],
    '11': ['Cz', '5', 'C4', '17'],
    'C4': ['11', '6', '13', '18'],
    '13': ['C4'],
    '14': ['C3', '15'],
    '15': ['14', '9', '16', '19'],
    '16': ['15', 'Cz', '17', 'Pz'],
    '17': ['16', '11', '18', '21'],
    '18': ['17', 'C4'],
    '19': ['15', 'Pz'],
    'Pz': ['19', '16', '21', '22'],
    '21': ['Pz', '17'],
    '22': ['Pz']
}

In [112]:
def spatial_filter_laplacian(raw):
    raw_arr = raw.get_data()
    transformed_timeseries = []
    for i in range(22):
        electrode_raw = raw_arr[i, :].copy()
        neighbors = LAPLACIAN_NEIGHBORS[raw.ch_names[i]]
        first_neighbor_idx = raw.ch_names.index(neighbors[0])
        neighboring_electrodes_avg = raw_arr[first_neighbor_idx, :].copy()
        for j in range(1, len(neighbors)):
            neighbor_idx = raw.ch_names.index(neighbors[j])
            neighboring_electrodes_avg += raw_arr[neighbor_idx, :].copy()
        neighboring_electrodes_avg /= len(neighbors)
        electrode_transformed = electrode_raw - neighboring_electrodes_avg
        transformed_timeseries.append(electrode_transformed)
    raw_filtered_arr = np.asarray(transformed_timeseries)
    raw_filtered_arr_with_eog = np.concatenate((raw_filtered_arr, raw_arr[22:, :]))
    raw_filtered = mne.io.RawArray(raw_filtered_arr_with_eog, raw.info)
    return raw_filtered

In [122]:
def bin_power_spectrum_1hz(psd, freqs):
    freqs_1hz_inc = list(range(4, 30))
    psds_binned_1hz = []
    curr_bin_freq = 4
    curr_bin_sum = 0
    curr_bin_size = 0
    for f in range(len(freqs)):
        if int(freqs[f]) > curr_bin_freq:
            psds_binned_1hz.append(curr_bin_sum / curr_bin_size)
            curr_bin_sum = 0
            curr_bin_size = 0
            curr_bin_freq += 1
        curr_bin_sum += psd[f]
        curr_bin_size += 1
    psds_binned_1hz.append(curr_bin_sum / curr_bin_size)

    return freqs_1hz_inc, psds_binned_1hz

In [123]:
def create_frequency_domain_features(epoch_data, info, normalize_band_power_within_trials=False):
    psds, freqs = mne.time_frequency.psd_array_multitaper(epoch_data, info['sfreq'], fmax=30, verbose=False)
    num_electrodes = len(psds)
    features_dict = {}
    for j in range(num_electrodes):
        freqs_binned, psd_binned = bin_power_spectrum_1hz(psds[j], freqs)
        if normalize_band_power_within_trials:
            psd_binned /= sum(psd_binned)
        for k in range(len(freqs_binned)):
            ch_name = info.ch_names[j]
            freq_bin_str = str(freqs_binned[k]) + 'Hz'
            band_power = psd_binned[k]
            feature_name = '{}_{}'.format(ch_name, freq_bin_str)
            features_dict[feature_name] = band_power
    return features_dict

In [8]:
def create_study_montage():
    montage_10_20 = mne.channels.make_standard_montage('standard_1020')
    data_10_20 = montage_10_20.get_positions()
    channels_used = 'Fz FC3 FC1 FCz FC2 FC4 C5 C3 C1 Cz C2 C4 C6 CP3 CP1 CPz CP2 CP4 P1 Pz P2 POz'
    ch_pos_filtered = OrderedDict()
    for ch in channels_used.split():
        ch_pos_filtered[ch] = data_10_20['ch_pos'][ch]
    study_montage = mne.channels.make_dig_montage(
        ch_pos=ch_pos_filtered, nasion=data_10_20['nasion'], lpa=data_10_20['lpa'], rpa=data_10_20['rpa'])
    study_montage.rename_channels({
        'FC3': '2', 'FC1': '3', 'FCz': '4', 'FC2': '5', 'FC4': '6',
        'C5': '7', 'C1': '9', 'C2': '11', 'C6': '13',
        'CP3': '14', 'CP1': '15', 'CPz': '16', 'CP2': '17', 'CP4': '18',
        'P1': '19', 'P2': '21',
        'POz': '22'
    })
    return study_montage

In [116]:
def prepare_subject_data(subject_no,train_or_test, baseline_normalize=False,
                         normalize_band_power_within_trials=True):
    # 1. Read file into an mne.Raw structure
 
    raw = mne.io.read_raw_gdf(
        get_data_path(subject_no, train_or_test), eog=['EOG-left', 'EOG-central', 'EOG-right'], preload=True
    )
    # 2. Rename annotations from numeric codes to readable descriptions
    raw.annotations.rename({
        '276': 'idle_eyes_open',
        '277': 'idle_eyes_closed',
        '768': 'trial_start',
        '769': 'cue_left',
        '770': 'cue_right',
        '771': 'cue_foot',
        '772': 'cue_tongue',
        # '783': 'cue_unknown',
        '1023': 'trial_rejected',
        '1072': 'idle_eye_movements',
        '32766': 'run_start'
    }).rename({'3idle_eyes_open6': 'run_start'});

    # 3. Rename EEG channels to match diagram from dataset description
    raw.rename_channels({
        'EEG-0': '2', 'EEG-1': '3', 'EEG-2': '4', 'EEG-3': '5', 'EEG-4': '6', 
        'EEG-5': '7', 'EEG-6': '9', 'EEG-7': '11', 'EEG-8': '13', 'EEG-9': '14',
        'EEG-10': '15', 'EEG-11': '16', 'EEG-12': '17', 'EEG-13': '18', 'EEG-14': '19',
        'EEG-15': '21', 'EEG-16': '22', 'EEG-Fz': 'Fz', 'EEG-C3': 'C3', 'EEG-Cz': 'Cz',
        'EEG-C4': 'C4', 'EEG-Pz': 'Pz'
    })

    # 4. Extract annotations into events, then remove annotations from Raw instance
    events, event_id = mne.events_from_annotations(raw)
    raw.set_annotations(None)
    event_id_inv = {v: k for k, v in event_id.items()}
    
    # 5. Apply bandpass filter
    raw.filter(4, 30, picks=['eeg'])
    
    # 6. Apply spatial filter (Laplacian Hjorth estimate)
    raw = spatial_filter_laplacian(raw)

    # 7. Epoch data (mne.Raw -> mne.Epochs)
    cue_event_ids = {e_id for e_name, e_id in event_id.items() if e_name.startswith('cue')}
    cue_events = np.array([event for event in events if event[2] in cue_event_ids])
    trial_event_id = {e_name: e_id for e_name, e_id in event_id.items() if e_name.startswith('cue')}
    baseline = (None, 0) if baseline_normalize else None
    epochs = mne.Epochs(raw, cue_events, event_id=trial_event_id, tmin=-2., tmax=4, baseline=baseline, preload=True)

    # 9. Set channel montage
    study_montage = create_study_montage()
    epochs.set_montage(study_montage)

    # 10. Remove trials marked as bad
    trial_rejected_events = events[events[:, 2] == event_id['trial_rejected']]
    rejected_epoch_start_sample_numbers = set(trial_rejected_events[:, 0])
    epoch_start_sample_numbers = events[events[:, 2] == event_id['trial_start']][:, 0]
    rejected_epoch_indices = [i for i in range(len(epoch_start_sample_numbers)) if epoch_start_sample_numbers[i] in rejected_epoch_start_sample_numbers]
    epochs.drop(rejected_epoch_indices)
    
    # 11. Split into X and y arrays
    X = epochs.get_data(picks=['eeg'], tmin=.5)
    y = [event_id_inv[epochs.events[i][2]] for i in range(len(epochs))]
    
    # 12. Fit a LabelEncoder
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    
    # 13. Transform into frequency domain features
    X = pd.DataFrame([create_frequency_domain_features(
        epoch_data, epochs.info, normalize_band_power_within_trials) for epoch_data in X])

    # 12. Split into train and test sets
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return epochs, cue_events, label_encoder, X, y

In [118]:
epochs, events, label_encoder, X, y = prepare_subject_data(2,"train", normalize_band_power_within_trials=True)

Extracting EDF parameters from D:\STUDY\SEM2 3rd-year\BCI\PJ\data2a\A02T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG
Creating raw.info structure...
Reading 0 ... 677168  =      0.000 ...  2708.672 secs...


C:\Users\buitr\anaconda3\lib\contextlib.py:126: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Used Annotations descriptions: ['cue_foot', 'cue_left', 'cue_right', 'cue_tongue', 'idle_eye_movements', 'idle_eyes_closed', 'idle_eyes_open', 'run_start', 'trial_rejected', 'trial_start']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 5 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 413 samples (1.652 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.4s finished


Creating RawArray with float64 data, n_channels=25, n_times=677169
    Range : 0 ... 677168 =      0.000 ...  2708.672 secs
Ready.
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 1501 original time points ...
0 bad epochs dropped
Dropped 18 epochs: 76, 82, 96, 98, 115, 159, 171, 183, 228, 238, 240, 241, 244, 254, 268, 274, 278, 279


In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train

,Fz_4Hz,Fz_5Hz,Fz_6Hz,Fz_7Hz,Fz_8Hz,Fz_9Hz,Fz_10Hz,Fz_11Hz,Fz_12Hz,Fz_13Hz,...,22_20Hz,22_21Hz,22_22Hz,22_23Hz,22_24Hz,22_25Hz,22_26Hz,22_27Hz,22_28Hz,22_29Hz
115,0.102024,0.189889,0.114143,0.065973,0.057343,0.090715,0.080042,0.037118,0.010333,0.016395,...,0.034810,0.016325,0.010537,0.009153,0.008844,0.009859,0.009534,0.007207,0.005975,0.006179
33,0.033649,0.172925,0.112346,0.066110,0.064735,0.070052,0.107475,0.087582,0.031865,0.021344,...,0.016908,0.017521,0.015084,0.010911,0.012210,0.007131,0.005511,0.008396,0.008889,0.005131
184,0.034114,0.137126,0.091139,0.070315,0.074317,0.102307,0.089643,0.079201,0.053850,0.025422,...,0.014269,0.015464,0.014112,0.007035,0.007215,0.010734,0.009898,0.006837,0.006510,0.004996
142,0.073195,0.212441,0.116160,0.073421,0.063523,0.059334,0.067512,0.052197,0.048185,0.040568,...,0.011890,0.006590,0.009997,0.016506,0.018875,0.019532,0.012447,0.005629,0.007903,0.008991
197,0.030211,0.088273,0.107906,0.097916,0.072519,0.082302,0.125754,0.079366,0.019625,0.020343,...,0.032866,0.021271,0.014417,0.015497,0.017162,0.012567,0.008204,0.006073,0.005238,0.003731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,0.060470,0.246361,0.161677,0.077898,0.057737,0.032480,0.046323,0.039217,0.018901,0.013674,...,0.030141,0.014298,0.008735,0.007239,0.008999,0.013826,0.013713,0.009498,0.006545,0.012032
188,0.086808,0.175463,0.109489,0.054757,0.037925,0.053635,0.082633,0.072072,0.030834,0.030774,...,0.017678,0.014026,0.019554,0.015535,0.009565,0.009175,0.007181,0.005069,0.007358,0.006970
71,0.045632,0.104916,0.069515,0.059946,0.078349,0.147416,0.146298,0.068096,0.030163,0.010274,...,0.012930,0.010973,0.004385,0.002997,0.003092,0.006620,0.008372,0.004662,0.001467,0.001282
106,0.030321,0.112727,0.124869,0.083891,0.096860,0.104315,0.079418,0.042222,0.023542,0.020393,...,0.012560,0.007110,0.003355,0.002855,0.002796,0.004266,0.003893,0.003411,0.003248,0.003167


In [88]:
clf = Pipeline([
    ('feature_selector', SelectKBest(score_func=chi2, k=50)), 
    ('classifier', LinearSVC(C=10, dual=False))
])

In [120]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [25]:
y_pred

array([2, 1, 0, 2, 2, 0, 1, 3, 2, 1, 0, 1, 3, 0, 3, 3, 2, 1, 2, 1, 2, 0,
       1, 3, 1, 3, 1, 1, 0, 0, 1, 0, 3, 1, 0, 2, 0, 3, 1, 0, 3, 2, 3, 2,
       0, 2, 1, 0, 1, 1, 2, 0, 2, 2], dtype=int64)

In [121]:
cv_results = cross_validate(clf, X_train, y_train, return_train_score=True)
cv_results

{'fit_time': array([0.01336551, 0.01047873, 0.01047921, 0.0104835 , 0.01047969]),
 'score_time': array([0.00300241, 0.00353956, 0.00299335, 0.00299191, 0.00299406]),
 'test_score': array([0.45454545, 0.55813953, 0.51162791, 0.60465116, 0.58139535]),
 'train_score': array([0.79069767, 0.78612717, 0.79768786, 0.79768786, 0.76878613])}

In [52]:
test_score= cv_results['test_score']
train_score = cv_results['train_score']

In [106]:
cv_results['train_score'].mean()

0.774290899314424

In [111]:
cv_results['test_score'].mean()

0.5373150105708244

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

test_scores = [0.88636364, 0.72727273, 0.72727273, 0.76744186, 0.8372093 ]
train_scores = [0.90804598, 0.9137931 , 0.90804598, 0.90857143, 0.90285714]

# Define the training set sizes
train_sizes = range(1, len(train_scores) + 1)

# Calculate the mean values
test_mean = np.mean(test_scores)
train_mean = np.mean(train_scores)

# Plot the scores
plt.plot(train_sizes, train_scores, 'o-', color="r", label="Train Score")
plt.plot(train_sizes, test_scores, 'o-', color="g", label="Test Score")

# Plot mean lines
plt.axhline(train_mean, color='r', linestyle='--', label='Train Mean')
plt.axhline(test_mean, color='g', linestyle='--', label='Test Mean')

plt.title("Training and Test Scores by Cross-Validate")
plt.xlabel("Training Set Size")
plt.ylabel("Score")
plt.legend(loc="best")
plt.grid(True)

plt.show()


In [ ]:
y_test

In [ ]:
remove_cue_prefix_from_labels = np.vectorize(lambda x: x[4:])
labels_no_cue = remove_cue_prefix_from_labels(label_encoder.inverse_transform(y_test))
labels_no_cue

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
y_true = y_test
cm = confusion_matrix(y_true, y_pred)

labels = ['Foot', 'Left', 'Right','Tounge']
# Plot confusion matrix
fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       xticklabels=labels, yticklabels=labels,
       xlabel='Predicted label',
       ylabel='True label',
       title='Confusion Matrix')

# Loop over data dimensions and create text annotations
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.show()
